In [1]:
import pandas as pd
import numpy as np
from  sklearn import preprocessing

In [2]:
train = np.load('../feature/nn_train.npy')
test = np.load('../feature/nn_test.npy')
train_weight = np.load('../feature/nn_weight_train.npy')
test_weight = np.load('../feature/nn_weight_test.npy')
train_person = np.load('../feature/nn_person_train.npy')
test_person = np.load('../feature/nn_person_test.npy')
train_mat1 = np.load('../feature/nn_mat1_train.npy')
test_mat1 = np.load('../feature/nn_mat1_test.npy')
train_mat2 = np.load('../feature/nn_mat2_train.npy')
test_mat2 = np.load('../feature/nn_mat2_test.npy')
# train_mat3 = np.load('../feature/nn_mat5_train.npy')
# test_mat3 = np.load('../feature/nn_mat5_test.npy')

In [4]:
# train = train[0:10000]
# test = test[0:10000]
# train_weight = train_weight[0:10000]
# test_weight = test_weight[0:10000]
# train_person = train_person[0:10000]
# test_person = test_person[0:10000]
# train_mat1 = train_mat1[0:10000]
# test_mat1 = test_mat1[0:10000]
# train_mat2 = train_mat2[0:10000]
# test_mat2 = test_mat2[0:10000]
# train_mat3 = train_mat3[0:10000]
# test_mat3 = test_mat3[0:10000]

In [3]:
train_mat2.shape

(2010000, 28)

In [4]:
train_mat = np.concatenate([train_mat1],1)
test_mat = np.concatenate([test_mat1],1)
del train_mat1
del test_mat1
train_person = np.concatenate([train_person,train_mat2],1)
test_person = np.concatenate([test_person,test_mat2],1)
del train_mat2
del test_mat2
# del train_mat3
# del test_mat3

In [6]:
# scaler = preprocessing.StandardScaler().fit(train_person)
# train_person = scaler.transform(train_person) 
# test_person = scaler.transform(test_person)

In [7]:
# train_weight = train_weight.reshape(len(train),45)
# test_weight = test_weight.reshape(len(test),45)
# scaler = preprocessing.StandardScaler().fit(train_weight)
# train_weight = scaler.transform(train_weight) 
# test_weight = scaler.transform(test_weight)

In [5]:
train_weight = train_weight.reshape(len(train),15,3)
test_weight = test_weight.reshape(len(test),15,3)

train_person = train_person.reshape(len(train),1,-1)
test_person = test_person.reshape(len(test),1,-1)


train_mat = train_mat.reshape(len(train),1,-1)
test_mat = test_mat.reshape(len(test),1,-1)

In [6]:
age_train_data = pd.read_csv('../data/age_train.csv',names=['uid','age_group'])
age_train_data['age_group'] = age_train_data['age_group'].apply(lambda x:x-1)
y_train = age_train_data['age_group'].values

# y_train = y_train[0:10000]

In [7]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)
from  sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

for i in range(train.shape[2]):
    scaler = preprocessing.StandardScaler().fit(train[:,:,i])
    train[:,:,i] = scaler.transform(train[:,:,i]) 
    test[:,:,i] = scaler.transform(test[:,:,i])
#     train[:,:,i] = min_max_scaler.fit_transform(train[:,:,i])
#     test[:,:,i] = min_max_scaler.transform(test[:,:,i])
for i in range(train_weight.shape[2]):
    scaler = preprocessing.StandardScaler().fit(train_weight[:,:,i])
    train_weight[:,:,i] = scaler.transform(train_weight[:,:,i]) 
    test_weight[:,:,i] = scaler.transform(test_weight[:,:,i])
#     train_weight[:,:,i] = min_max_scaler.fit_transform(train_weight[:,:,i])
#     test_weight[:,:,i] = min_max_scaler.transform(test_weight[:,:,i])
for i in range(train_person.shape[2]):
    scaler = preprocessing.StandardScaler().fit(train_person[:,:,i])
    train_person[:,:,i] = scaler.transform(train_person[:,:,i]) 
    test_person[:,:,i] = scaler.transform(test_person[:,:,i])


In [8]:
from torch.utils.data import DataLoader,Dataset
class Getdata(Dataset):
    def __init__(self,feature,weight,person,mat1,labels,transform=preprocessing.normalize):
        self.feature = feature
        self.weight = weight
        self.person = person
        self.mat1 = mat1
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.feature)
    def __getitem__(self, item):
        sample={}
        if self.labels is not None:
            sample['label'] = self.labels[item]
            sample['feature'] = self.transform(self.feature[item],norm='l2')
            sample['weight'] = self.transform(self.weight[item],norm='l2')
            sample['person'] = self.transform(self.person[item],norm='l2')
            sample['mat1'] = self.mat1[item]
        return sample

class Gettestdata(Dataset):
    def __init__(self,feature,weight,person,mat1,transform=preprocessing.normalize):
        self.feature = feature
        self.weight = weight
        self.person = person
        self.mat1 = mat1
        self.transform = transform
    def __len__(self):
        return len(self.feature)
    def __getitem__(self, item):
        sample={}
        if self.feature is not None:
            sample['feature'] = self.transform(self.feature[item],norm='l2')
            sample['weight'] = self.transform(self.weight[item],norm='l2')
            sample['person'] = self.transform(self.person[item],norm='l2')
            sample['mat1'] = self.mat1[item]
        return sample


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Mymodel(nn.Module):
    def __init__(self,hidden_dim1=666,hidden_dim2=100,hidden_dim3=6):
        super(Mymodel,self).__init__()
        self.mat1cnn = nn.Sequential(*[
            nn.Conv2d(1,16,(1,3),stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
#             nn.MaxPool2d(1,1),
            nn.Conv2d(16,32,(1,3),stride=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
#             nn.MaxPool2d(1,1),
            nn.Conv2d(32,64,(1,5),stride=5),
            nn.BatchNorm2d(64),
            nn.ReLU(),
#             nn.MaxPool2d(1,1),
        ])
        self.modelperson = nn.Sequential(*[
            nn.Linear(1856, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
        ])
        self.personcnn = nn.Sequential(*[
            nn.Conv2d(1,16,(1,3),stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
#             nn.MaxPool2d(1,1),
            nn.Conv2d(16,32,(1,3),stride=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
#             nn.MaxPool2d(1,1),      
        ])
        self.featurecnn = nn.Sequential(*[
            nn.Conv2d(1,16,2,2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(1,1),
            nn.Conv2d(16,32,3,3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(1,1),
        ])
    def forward(self,fea_weight,person,mat1):
        fea_weight = fea_weight.float()
        person = person.float()
        person = person.unsqueeze(1)
        mat1 = mat1.float()
        mat1 = self.mat1cnn(mat1)
        mat1 = mat1.view(mat1.shape[0],-1)
        fea_weight = fea_weight.unsqueeze(1)
        fea_weight = self.featurecnn(fea_weight)
        fea_weight = fea_weight.view(fea_weight.shape[0],-1)
        person = self.personcnn(person)
        person = person.view(person.shape[0],-1)
        allfeature = torch.cat((mat1,fea_weight,person),1)
        end = self.modelperson(allfeature)
        return end


In [10]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [11]:
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
np.random.seed(666)
torch.manual_seed(666)
torch.cuda.manual_seed_all(666)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, y_train)):
    print(fold_)
    if fold_==1:
        break
    model = Mymodel()
    model = model.to(device)
    train_loaders = DataLoader(Getdata(train[trn_idx],train_weight[trn_idx],
                                       train_person[trn_idx],train_mat[trn_idx],y_train[trn_idx]),batch_size=64,shuffle=True)
    val_loaders = DataLoader(Getdata(train[val_idx],train_weight[val_idx],
                                      train_person[val_idx],train_mat[val_idx],y_train[val_idx]),batch_size=64,shuffle=True)
    test_loaders = DataLoader(Gettestdata(test,test_weight,test_person,test_mat),batch_size=64)
    n_ep = 4
    wd = 1e-5
    lr = 1e-4
    criterion =nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=wd, lr=lr)
    for i_ep in range(n_ep):
        for _,sample in tqdm(enumerate(train_loaders),total=len(train_loaders)):
            feature = sample['feature'].to(device)
#             feature = feature.unsqueeze(1)
            weight = sample['weight'].to(device)
            fea_weight = torch.cat((feature,weight),2)
            person = sample['person'].to(device)
#             person = person.unsqueeze(1)
            mat1 = sample['mat1'].to(device)
            mat1 = mat1.unsqueeze(1)
            label = sample['label'].to(device)
            optimizer.zero_grad()
            result = model.forward(fea_weight,person,mat1)
            loss = criterion(result,label)
            loss.backward()
            optimizer.step()
        with torch.no_grad():
            valmodel = model.eval()
            losses = []
            labels = []
            preds = []
            for _,sample_val in enumerate(val_loaders):
                feature_val = sample_val['feature'].to(device)
#                 feature_val = feature_val.unsqueeze(1)
                weight_val = sample_val['weight'].to(device)
                fea_weight_val = torch.cat((feature_val,weight_val),2)
                person_val = sample_val['person'].to(device)
#                 person_val = person_val.unsqueeze(1)
                mat1_val = sample_val['mat1'].to(device)
                mat1_val = mat1_val.unsqueeze(1)
                label_val = sample_val['label'].to(device)
                result_val = valmodel.forward(fea_weight_val,person_val,mat1_val)
#                 print(np.argmax(result_test.numpy(),axis=1))
                preds += [np.argmax(result_val.detach().cpu().numpy(),axis=1)]
                labels += [label_val.detach().cpu().numpy()]
            preds = np.concatenate(preds, 0)
            labels = np.concatenate(labels, 0)
            print(accuracy_score(labels,preds))
    with torch.no_grad():
        testmodel = model.eval()
        preds_test = []
        for _,sample_test in tqdm(enumerate(test_loaders),total=len(test_loaders)):
            feature_test = sample_test['feature'].to(device)
#             feature_test = feature_test.unsqueeze(1)
            weight_test = sample_test['weight'].to(device)
            fea_weight_test = torch.cat((feature_test,weight_test),2)
            person_test = sample_test['person'].to(device)
#             person_test = person_test.unsqueeze(1)
            mat1_test = sample_test['mat1'].to(device)
            mat1_test = mat1_test.unsqueeze(1)
            result_test = testmodel.forward(fea_weight_test,person_test,mat1_test)
            preds_test += [np.argmax(result_test.detach().cpu().numpy(),axis=1)]
        predictions = np.concatenate(preds_test, 0)
        print(predictions)





0


  0%|          | 1/25125 [00:00<54:49,  7.64it/s]

0.6050149253731343


  1%|          | 187/25125 [00:07<17:10, 24.20it/s]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-2fbcf1972322>", line 26, in <module>
    for _,sample in tqdm(enumerate(train_loaders),total=len(train_loaders)):
  File "/home/lzy/.local/lib/python3.6/site-packages/tqdm/_tqdm.py", line 933, in __iter__
    for obj in iterable:
  File "/home/lzy/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 615, in __next__
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "/home/lzy/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 615, in <listcomp>
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "<ipython-input-8-4341f8ee5248>", line 17, in __getitem__
    sample['weight'] = self.transform(self.weight[item],norm='l2')
  File "/home/lzy/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.

KeyboardInterrupt: 

In [ ]:
del train_mat
del test_mat

In [16]:
predictions.shape

(502500,)

In [17]:
age_test_data = pd.read_csv('../data/age_test.csv',names=['id'])

In [18]:
age_test_data['label'] = predictions

In [19]:
age_test_data['label'] = age_test_data['label'].apply(lambda x:round(x+1))

In [20]:
age_test_data[['id','label']].to_csv('../submit/submission.csv',index=False)